In [651]:
! pip install xgboost

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [652]:
import json 
from striprtf.striprtf import rtf_to_text
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline 
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_absolute_error,root_mean_squared_error

In [653]:
import json 
from striprtf.striprtf import rtf_to_text

def load_json():

    try:
        #file_path=input("Please Enter file Path")
        #file_path= r"C:\Desktop\Data science\Screening Test - DS\algoparams_from_ui.json.rtf"
        #file_path=r"C:\Desktop\Data science\Screening Test - DS\JSON_file.json"
        file_path=r"C:\Desktop\Data science\Screening Test - DS\algoparams_from_ui.json"
        #file_path=  r"C:\Users\nikit\Downloads\feature_reduction.json"
        m=file_path
        type=m.split('.')[-1]
        print(type)
        
        if(type=='rtf'):
            with open(file_path,'r') as f:
                content=f.read()
                text=rtf_to_text(content)
                d=json.loads(text)
                print(json.dumps(d,indent=4))
            return d
        elif(type=='json'):
            with open(file_path,'r') as f:
                d=json.load(f)
                print(json.dumps(d,indent=4))
            return d
        else:
            print("File_path is not correct ")
            return 
    except FileNotFoundError:
        print("File not found ")
    

In [654]:
d=load_json()
d

json
{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {
        "session_info": {
            "project_id": "1",
            "experiment_id": "kkkk-11",
            "dataset": "iris_modified.csv",
            "session_name": "test",
            "session_description": "test"
        },
        "target": {
            "prediction_type": "Regression",
            "target": "petal_width",
            "type": "regression",
            "partitioning": true
        },
        "train": {
            "policy": "Split the dataset",
            "time_variable": "sepal_length",
            "sampling_method": "No sampling(hole data)",
            "split": "Randomly",
            "k_fold": false,
            "train_ratio": 0.2,
            "random_seed": 0
        },
        "metrics": {
            "optomize_model_hyperparameters_for": "AUC",
            "optimize_threshold_for": "F1 Score",
            "compute_lift_at": 0,
            "cost_matrix_gain_for

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(hole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0.2,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name

In [655]:
def load_data():
  df=pd.read_csv('iris.csv')
  return df
df=load_data()

In [656]:
t=d['design_state_data']['train']
t


{'policy': 'Split the dataset',
 'time_variable': 'sepal_length',
 'sampling_method': 'No sampling(hole data)',
 'split': 'Randomly',
 'k_fold': False,
 'train_ratio': 0.2,
 'random_seed': 0}

In [657]:
def train():
    t=d['design_state_data']['train']

    X_train, X_test, y_train, y_test = train_test_split(df.drop(['petal_width'],axis=1), 
                                                        df['petal_width'], test_size=0.3,random_state=t['random_seed'])
    a=y_train.name
        
    return X_train, X_test, y_train, y_test,a

In [658]:
X_train, X_test, y_train, y_test,y_train_name=train()
y_train_name

'petal_width'

In [659]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline 
from sklearn.compose import ColumnTransformer,make_column_transformer


def get_transformer_impute():
    col_handle_d=d['design_state_data']['feature_handling']
    col_transormer=[]
    l=[]
    for i in col_handle_d:
        if col_handle_d[i]['is_selected']==True and col_handle_d[i].get('feature_details', {}).get('missing_values')=='Impute':
            if(col_handle_d[i]['feature_name']==y_train.name):
                continue
            else:      
                if col_handle_d[i]['feature_details']['impute_with'].lower()=='average of values':
                    l.append(X_train.columns.get_loc(col_handle_d[i]['feature_name']))
                else:
                    cus=col_handle_d[i].get('feature_details', {}).get('impute_value')
                    col_transormer.append((SimpleImputer(strategy = 'constant',fill_value=cus),[X_train.columns.get_loc(col_handle_d[i]['feature_name'])]))
        else:
            if col_handle_d[i]['is_selected']==True and col_handle_d[i]['feature_details']['text_handling'].lower()=='tokenize and hash':
                    n=col_handle_d['species']['feature_details']['hash_columns']
                    col_transormer.append((HashingVectorizer(n_features=1),X_train.columns.get_loc(col_handle_d[i]['feature_name'])))
            


    col_transormer.append((SimpleImputer(),l))
    return col_transormer

transformer=get_transformer_impute()

pipe1_impute=make_column_transformer(*transformer)
pipe1_impute

ColumnTransformer(transformers=[('simpleimputer-1',
                                 SimpleImputer(fill_value=-1,
                                               strategy='constant'),
                                 [1]),
                                ('hashingvectorizer',
                                 HashingVectorizer(n_features=1), 3),
                                ('simpleimputer-2', SimpleImputer(), [0, 2])])

In [660]:
def prepro_y():
    X_train, X_test, y_train, y_test,y_train_name=train()
    a=d['design_state_data']['feature_handling'][y_train_name]['feature_details']
    if a['missing_values']=='Impute':
        if a['impute_with']=='custom':
            si=SimpleImputer(strategy='constant',fill_value=a['impute_value'])
            y_train=si.fit_transform(y_train.values.reshape(-1,1))
            y_test=si.fit_transform(y_test.values.reshape(-1,1))

            return pd.Series(y_train.flatten()),pd.Series(y_test.flatten())
        else:
            si=SimpleImputer(strategy='mean')
            y_train=si.fit_transform(y_train.values.reshape(-1,1))
            y_test=si.fit_transform(y_test.values.reshape(-1,1))

            return pd.Series(y_train.flatten()),pd.Series(y_test.flatten())




In [661]:
a=d['design_state_data']['feature_handling'][y_train.name]['feature_details']
a.get('missing_values')

'Impute'

In [662]:
y_train,y_test=prepro_y()

In [663]:
def get_transformer_reduction():
    X_train, X_test, y_train, y_test,y_train_name=train()
    red_method=d['design_state_data']['feature_reduction']
    l=[]
    for i,j in d['design_state_data']['feature_handling'].items():
        if j.get('is_selected')==True:
            if(j['feature_name']==y_train.name):
                continue
            else:
                l.append(X_train.columns.get_loc(j['feature_name']))
        

    if(red_method['feature_reduction_method'].lower()=='tree-based'):
        
        pipe2_tranformer=make_column_transformer(
            
                (
                    SelectFromModel(
                        RandomForestRegressor(
                            n_estimators=int(red_method['num_of_trees']),
                            max_depth=int(red_method['depth_of_trees'])
                            ),
                            max_features=int(red_method['num_of_features_to_keep']),
                            threshold=-np.inf
                    ),
                    l
                    )
           
        )
                    
        return pipe2_tranformer
    
    elif red_method['feature_reduction_method'].lower()=='principal component analysis':
        pipe2_tranformer=make_column_transformer(
            (
            PCA(n_components=red_method['num_of_features_to_keep']),
            l
            )
            
        )
        return pipe2_tranformer
    elif red_method['feature_reduction_method'].lower()=='correlation with target':
        pipe2_tranformer=make_column_transformer(
            (
                SelectKBest(score_func=f_regression,k=red_method['num_of_features_to_keep']),
                l
            )

        )
        return pipe2_tranformer
    else:
        return 
        
    


pipe2_tranformer= get_transformer_reduction()
pipe2_tranformer

ColumnTransformer(transformers=[('selectfrommodel',
                                 SelectFromModel(estimator=RandomForestRegressor(max_depth=6,
                                                                                 n_estimators=5),
                                                 max_features=4,
                                                 threshold=-inf),
                                 [0, 1, 2, 3])])

In [664]:
def choose_Algo():
    n=d['design_state_data']['algorithms']
    if n['RandomForestRegressor']['is_selected']==True :
        if n['RandomForestRegressor']['feature_sampling_statergy']=='Default':
                m=1.0
        else:
              m=n['RandomForestRegressor']['feature_sampling_statergy']
        if n['RandomForestRegressor']['parallelism']==0:
                p=-1
        else:
              p=n['RandomForestRegressor']['parallelism']
        algo=RandomForestRegressor(
            n_estimators=int((n['RandomForestRegressor']['min_trees']+n['RandomForestRegressor']['max_trees'])//2),
            max_features=m,
            max_depth= int((n['RandomForestRegressor']['min_depth']+n['RandomForestRegressor']['max_depth'])//2),
            min_samples_leaf=int((n['RandomForestRegressor']['min_samples_per_leaf_min_value']+n['RandomForestRegressor']['min_samples_per_leaf_max_value'])//2),
            n_jobs=p

        )
        return algo
    elif n['GBTRegressor']['is_selected']==True:
          GBT=d['design_state_data']['algorithms']['GBTRegressor']
          if GBT['use_deviance']==True:
                m='squared_error'
                
          algo=GradientBoostingRegressor(
                n_estimators=int((GBT['num_of_BoostingStages'][0]+GBT['num_of_BoostingStages'][1])//2),
                loss=m,
                subsample=int((GBT['min_subsample']+GBT['max_subsample'])//2),
                learning_rate=(GBT['min_stepsize']+GBT['max_stepsize'])/2,
                max_depth=int((GBT['min_depth']+GBT['max_depth'])//2),    
          )
          
          return algo
    elif n['LinearRegression']['is_selected']==True:
          lin=d['design_state_data']['algorithms']['LinearRegression']

          algo=LinearRegression(
                n_jobs=lin['parallelism']
          )
          return algo
    elif n['RidgeRegression']['is_selected']==True:
          rid=d['design_state_data']['algorithms']['RidgeRegression']

          algo=Ridge(
                alpha=rid['regularization_term'],
                max_iter=int((rid['min_iter']+rid['max_iter'])//2)
          )


          return algo
    
    elif n['LassoRegression']['is_selected']==True:
          rid=d['design_state_data']['algorithms']['LassoRegression']
          
          algo=Lasso(
                alpha=rid['regularization_term'],
                max_iter=int((rid['min_iter']+rid['max_iter'])//2)
          )

     
          return algo
    
    elif n['ElasticNetRegression']['is_selected']==True:
          rid=d['design_state_data']['algorithms']['ElasticNetRegression']
          if type(rid['regularization_term'])==str:
                m=0.9
          else:
                m=rid['regularization_term']

          algo=ElasticNet(
                alpha=m,
                max_iter=int((rid['min_iter']+rid['max_iter'])//2),
                l1_ratio=(rid['min_elasticnet']+rid['max_elasticnet'])/2
          )

          
          return algo
    
    elif n['xg_boost']['is_selected']==True:
          rid=d['design_state_data']['algorithms']['xg_boost']
          k=rid.keys()
          if('dart' in k==True):
            m='dart'
          elif 'gbtree'in k==True:
            m='gbtree'
          elif 'gblinear' in k ==True:
            m='gblinear'   
          else:
            m='dart'
   
          algo=XGBRegressor(
                booster=m,
                random_state=rid['random_state'],
                n_estimator=rid['max_num_of_trees'],
                max_depth=int((rid['max_depth_of_tree'][0]+rid['max_depth_of_tree'][0])//2),
                learning_rate=(rid['learningRate'][0]+rid['learningRate'][1])/2,
                reg_alpha=rid['l1_regularization'][0],
                reg_lambda=rid['l2_regularization'][0],
                gamma=rid['gamma'][0],
                min_child_weight=rid['min_child_weight'][0],
                subsample=rid['sub_sample'][0],
                colsample_bytree=rid['col_sample_by_tree'][0],
                njobs=rid['parallelism']            
          )

          
          return algo
    
    elif n['DecisionTreeRegressor']['is_selected']==True:
         rid=d['design_state_data']['algorithms']['DecisionTreeRegressor'] 
         if rid['use_random']==True:
              m="random"
         else:
              m='best'

         algo=DecisionTreeRegressor(
              max_depth=rid['max_depth'],
              min_samples_split=rid['min_depth'],
              min_samples_leaf=(rid['min_samples_per_leaf'][0]+rid['min_samples_per_leaf'][1])//2,
              splitter=m
              )
         
         return algo
    
    elif n['SVM']['is_selected']:
         rid=d['design_state_data']['algorithms']['SVM']
         if rid['linear_kernel']==True:
              m='linear'
         elif rid['polynomial_kernel']==True:
              m='poly'
         elif rid['rep_kernel']==True:
              m='rbf'
         elif rid['sigmoid_kernel']==True:
              m='sigmoid'
         else:
              m='rbf'
         if rid['auto']==True:
              k='auto'
         elif rid['scale']==True:
              k='scale'
         elif rid['custom_gamma_values']==True:
              k=0.7
         else:
              k='scale'
         


         algo=SVR(
              kernel=m,
              C=(rid['c_value'][0]+rid['c_value'][1])/2,
              gamma=k,
              tol=rid['tolerance'],
              max_iter=rid['max_iterations'],
              

         )

         
         return algo
    
    elif n['SGD']['is_selected']:
         rid=d['design_state_data']['algorithms']['SGD']
         if rid['use_modified_hubber_loss']==True:
              m='huber'
         else:
              m='squared_error'
         if rid['use_l1_regularization']=='on':
              j='l1'
         elif rid['use_l2_regularization']=='on':
              j='l2'
         elif rid['use_elastic_net_regularization']:
              j='elasticnet'
         else: 
              j='l2'


         algo=SGDRegressor(
              loss=m,
              tol=rid['tolerance'],
              penalty=j,
              alpha=rid['alpha_value'][0]    
         )

         
         return algo
         
    elif n['KNN']['is_selected']:
         rid=d['design_state_data']['algorithms']['KNN']
         if rid['distance_weighting']==True:
              k='distance'
         else:
              k='uniform'
         algo=KNeighborsRegressor(
              n_neighbors=rid['k_value'][0],
              weights=k,
              p=rid['p_value']
         )

        
         return algo
    
    elif n['extra_random_trees']['is_selected']:
         rid=d['design_state_data']['algorithms']['extra_random_trees']
         
         algo=ExtraTreesRegressor(
               max_depth=int((rid['max_depth'][0]+rid['max_depth'][1])//2),
               min_samples_leaf=int((rid['min_samples_per_leaf'][0]+rid['min_samples_per_leaf'][1])//2)  

         )

         
         return algo
    
    elif n['neural_network']['is_selected']:
         rid=d['design_state_data']['algorithms']['neural_network']
         if rid['activation']=='relu':
              m='relu'
         elif rid['activation']=='tanh':
              m='tanh'
         elif rid['activation']=='logistic':
              m='logistic'
         elif rid['activation']=='identity':
              m='identity'
         else:
              m='relu'

         if rid['solver']=='lbfgs':
              k='lbfgs'
         elif rid['solver']=='sgd':
              k='sgd' 
         elif rid['solver']=='adam':
              k='adam' 
         else:
              k='adam'    

         algo=MLPRegressor(
              hidden_layer_sizes=rid['hidden_layer_sizes'],
              activation=m,
              alpha=rid['alpha_value'],
              max_iter=rid['max_iterations'],
              tol=rid['convergence_tolerance'],
              early_stopping=rid['early_stopping'],
              solver=k,
              shuffle=rid['shuffle_data'],
              learning_rate_init=rid['initial_learning_rate'] ,
              beta_1=rid['beta_1'],
              beta_2=rid['beta_2'],
              epsilon=rid['epsilon'],
              nesterovs_momentum=rid['use_nesterov_momentum']

              
         )

         
         return algo
            
    else:
          return      
   

    
algo=choose_Algo()
type(algo)    

sklearn.neural_network._multilayer_perceptron.MLPRegressor

In [665]:
pipe=make_pipeline(pipe1_impute,pipe2_tranformer,algo)
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='constant'),
                                                  [1]),
                                                 ('hashingvectorizer',
                                                  HashingVectorizer(n_features=1),
                                                  3),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(), [0, 2])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('selectfrommodel',
                                                  SelectFromModel(estimator=RandomForestRegressor(max_depth=6,
                                                                                                  n_estimators=5),
                                                                  max_features=4,
                                                                  threshold=-inf),
                                                  [0, 1, 2, 3])])),
                ('mlpregressor',
                 MLPRegressor(alpha=0, beta_1=0, beta_2=0, early_stopping=True,
                              epsilon=0.002, hidden_layer_sizes=[67, 89],
                              learning_rate_init=1e-05, max_iter=100,
                              nesterovs_momentum=False, tol=0))])

In [666]:
pipe.fit(X_train,y_train)

C:\Users\nikit\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='constant'),
                                                  [1]),
                                                 ('hashingvectorizer',
                                                  HashingVectorizer(n_features=1),
                                                  3),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(), [0, 2])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('selectfrommodel',
                                                  SelectFromModel(estimator=RandomForestRegressor(max_depth=6,
                                                                                                  n_estimators=5),
                                                                  max_features=4,
                                                                  threshold=-inf),
                                                  [0, 1, 2, 3])])),
                ('mlpregressor',
                 MLPRegressor(alpha=0, beta_1=0, beta_2=0, early_stopping=True,
                              epsilon=0.002, hidden_layer_sizes=[67, 89],
                              learning_rate_init=1e-05, max_iter=100,
                              nesterovs_momentum=False, tol=0))])

In [667]:
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

-1.6392743967922891

In [668]:
y_train

0      1.0
1      1.8
2      2.5
3      1.5
4      1.8
      ... 
100    0.1
101    1.8
102    1.0
103    2.2
104    0.2
Length: 105, dtype: float64

## Grid Search CV

In [669]:
def algoname():
    alg=[]
    for i,j in pipe.steps:
        alg.append(type(j).__name__)

    return alg[-1]


In [670]:
def parameter():
    alg=algoname()

    if alg=='RandomForestRegressor':
        pm={
            'randomforestregressor__n_estimators':[300,400,500],
            'randomforestregressor__max_depth':[1000,2000],
            'randomforestregressor__min_samples_split':[5,7],
            'randomforestregressor__min_samples_leaf':[1,2,4]
        }

        return pm
    elif alg=='GradientBoostingRegressor':
        pm={
            'gradientboostingregressor__n_estimators':[50,100,200],
            'gradientboostingregressor__learning_rate':[0.01,0.1,0.2],
            'gradientboostingregressor__subsample':[0.8,1.0],
            'gradientboostingregressor__max_depth':[3,5,7]
        }
        return pm
    elif alg=="Ridge":
        pm={
           'ridge__alpha':[0.1,1.0,10],
           'ridge__max_iter':[1000,2000] 
        }
        return pm
    elif alg=='LinearRegression':
        pm={
             "linearregression__fit_intercept": [True, False]
        }
        return pm
    elif alg=='Lasso':
        pm={
           'lasso__alpha':[0.01,0.1,1.0,10.0],
           'lasso__max_iter':[1000,2000,3000] 
        }

        return pm
    elif alg=='ElasticNet':
        pm={
           'elasticnet__alpha':[0.01,0.1,10.0],
           'elasticnet__l1_ratio':[0.2,0.5,0.8,1.0],
           'elasticnet__max_iter':[1000,2000,3000] 
        }

        return pm  
    elif alg=='XGBRegressor':
        pm={
            'xgbregressor__n_estimators':[50,100,200],
            'xgbregressor__max_depth':[3,5,7],
            'xgbregressor__learning_rate':[0.01,0.1,0.2],
            'xgbregressor__subsample':[0.8,1.0],
            
        } 
        return pm
    elif alg=='DecisionTreeRegressor':
        pm={
          'decisiontreeregressor__max_depth':[10,20,30],
          'decisiontreeregressor__min_samples_split':[2,5,10],
          'decisiontreeregressor__min_samples_leaf':[1,2,4],
          'decisiontreeregressor__splitter':['best','random'] 
        }
        return pm 
    elif alg=='SVR':
        pm={
            'svr__kernel':['linear','rbf','poly','sigmoid'],
            'svr__C':[0.1,1.0,10.0],
            'svr__gamma':['scale','auto',0.1,1.0],
        
        }
        return pm
    elif alg=='SGDRegressor':
        pm={
            'sgdregressor__loss':['squared_error','hubber'],
            'sgdregressor__penalty':['l1','l2','elasticnet'],
            'sgdregressor__alpha':[0.0001,0.001,0.01],
            'sgdregressor__tol':[1e-3,1e-4]
        }
        return pm
    elif alg=='KNeighborsRegressor':
        pm={
            'kneighborsregressor__n_neighbors':[3,5,10,20],
            'kneighborsregressor__weights':['uniform','distance'],
            'kneighborsregressor__p':[1,2]
        }
        return pm
    elif alg=='ExtraTreesRegressor':
        pm={
            'extratreesregressor__min_samples_leaf':[1,2,4],
            'extratreesregressor__max_depth':[10,20,30]
        }
        return pm
    elif alg=='MLPRegressor':
        pm={
            'mlpregressor__hidden_layer_sizes':[(50,),(100,),(50,50),(100,50)],
            'mlpregressor__activation':['relu','tanh','logistic'],
            'mlpregressor__solver':['lbfgs','adam','sgd'],
            'mlpregressor__alpha':[0.0001,0.001,0.01],
            'mlpregressor__learning_rate_init':[0.001,0.01,0.1],
            
        }
        return pm

type(parameter())

dict

In [671]:
grid_search=GridSearchCV(
        estimator=pipe,
        param_grid=parameter(),
        cv=d['design_state_data']['hyperparameters']['num_of_folds'],
        n_jobs=d['design_state_data']['hyperparameters']['parallelism'],
        scoring='r2'
    )

In [672]:
grid_search.fit(X_train,y_train)


C:\Users\nikit\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(transformers=[('simpleimputer-1',
                                                                         SimpleImputer(fill_value=-1,
                                                                                       strategy='constant'),
                                                                         [1]),
                                                                        ('hashingvectorizer',
                                                                         HashingVectorizer(n_features=1),
                                                                         3),
                                                                        ('simpleimputer-2',
                                                                         SimpleImputer(),
                                                                         [0,
                                                                          2])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(transformers=[('selectfrommodel',
                                                                         Sel...
                                                     learning_rate_init=1e-05,
                                                     max_iter=100,
                                                     nesterovs_momentum=False,
                                                     tol=0))]),
             n_jobs=5,
             param_grid={'mlpregressor__activation': ['relu', 'tanh',
                                                      'logistic'],
                         'mlpregressor__alpha': [0.0001, 0.001, 0.01],
                         'mlpregressor__hidden_layer_sizes': [(50,), (100,),
                                                              (50, 50),
                                                              (100, 50)],
                         'mlpregressor__learning_rate_init': [0.001, 0.01, 0.1],
                         'mlpregressor__solver': ['lbfgs', 'adam', 'sgd']},
             scoring='r2')

## Best Score

In [673]:
grid_search.best_score_

np.float64(0.9396042158369527)

## Best Parameter

In [674]:
grid_search.best_params_

{'mlpregressor__activation': 'relu',
 'mlpregressor__alpha': 0.001,
 'mlpregressor__hidden_layer_sizes': (50, 50),
 'mlpregressor__learning_rate_init': 0.1,
 'mlpregressor__solver': 'lbfgs'}

### mean_absolute_error

In [675]:
mean_absolute_error(y_test,y_pred)

np.float64(0.9515318590098418)

### mean_squared_error

In [676]:
mean_squared_error(y_test,y_pred)

np.float64(1.3489624694716147)

### root_mean_squared_error

In [677]:
root_mean_squared_error(y_test,y_pred)

np.float64(1.161448435993443)

### r2_score

In [678]:
r2_score(y_test,y_pred)

-1.6392743967922891